In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS').getOrCreate()

In [2]:
# create correct data schema.
from pyspark.sql.types import *
schema = StructType([StructField('No',IntegerType(),True),
               StructField('year',IntegerType(),True),
               StructField('month',IntegerType(),True),
               StructField('day',IntegerType(),True),
               StructField('hour',IntegerType(),True),
               StructField('PM25',FloatType(),True),
               StructField('PM10',FloatType(),True),
               StructField('SO2',FloatType(),True),
               StructField('NO2',FloatType(),True),
               StructField('CO',FloatType(),True),
               StructField('O3',FloatType(),True),
               StructField('TEMP',FloatType(),True),
               StructField('PRES',FloatType(),True),
               StructField('DewPointTempeature',FloatType(),True),
               StructField('RAIN',FloatType(),True),
               StructField('WindDirection',StringType(),True),
               StructField('WindSpend',FloatType(),True),
               StructField('station',StringType(),True)])

In [3]:
df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .load("../DataSet/PRSA_Data_Aotizhongxin_20130301-20170228.csv")
df_with_schema.show()

+---+----+-----+---+----+-----+----+---+---+---+---+----+------+-----+----+---+----+------------+
| No|year|month|day|hour|PM2.5|PM10|SO2|NO2| CO| O3|TEMP|  PRES| DEWP|RAIN| wd|WSPM|     station|
+---+----+-----+---+----+-----+----+---+---+---+---+----+------+-----+----+---+----+------------+
|  1|2013|    3|  1|   0|    4|   4|  4|  7|300| 77|-0.7|  1023|-18.8|   0|NNW| 4.4|Aotizhongxin|
|  2|2013|    3|  1|   1|    8|   8|  4|  7|300| 77|-1.1|1023.2|-18.2|   0|  N| 4.7|Aotizhongxin|
|  3|2013|    3|  1|   2|    7|   7|  5| 10|300| 73|-1.1|1023.5|-18.2|   0|NNW| 5.6|Aotizhongxin|
|  4|2013|    3|  1|   3|    6|   6| 11| 11|300| 72|-1.4|1024.5|-19.4|   0| NW| 3.1|Aotizhongxin|
|  5|2013|    3|  1|   4|    3|   3| 12| 12|300| 72|  -2|1025.2|-19.5|   0|  N|   2|Aotizhongxin|
|  6|2013|    3|  1|   5|    5|   5| 18| 18|400| 66|-2.2|1025.6|-19.6|   0|  N| 3.7|Aotizhongxin|
|  7|2013|    3|  1|   6|    3|   3| 18| 32|500| 50|-2.6|1026.5|-19.1|   0|NNE| 2.5|Aotizhongxin|
|  8|2013|    3|  1|

In [4]:
df_with_schema.printSchema()

root
 |-- No: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- PM2.5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- SO2: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- CO: string (nullable = true)
 |-- O3: string (nullable = true)
 |-- TEMP: string (nullable = true)
 |-- PRES: string (nullable = true)
 |-- DEWP: string (nullable = true)
 |-- RAIN: string (nullable = true)
 |-- wd: string (nullable = true)
 |-- WSPM: string (nullable = true)
 |-- station: string (nullable = true)



In [5]:
new_rename_df = df_with_schema.withColumnRenamed("DEWP","DewPointTempeature") \
    .withColumnRenamed("wd","WindDirection")\
    .withColumnRenamed("WSPM","WindSpend")\
    .withColumnRenamed("PM2.5","PM25")
new_rename_df.show()

+---+----+-----+---+----+----+----+---+---+---+---+----+------+------------------+----+-------------+---------+------------+
| No|year|month|day|hour|PM25|PM10|SO2|NO2| CO| O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+---+----+-----+---+----+----+----+---+---+---+---+----+------+------------------+----+-------------+---------+------------+
|  1|2013|    3|  1|   0|   4|   4|  4|  7|300| 77|-0.7|  1023|             -18.8|   0|          NNW|      4.4|Aotizhongxin|
|  2|2013|    3|  1|   1|   8|   8|  4|  7|300| 77|-1.1|1023.2|             -18.2|   0|            N|      4.7|Aotizhongxin|
|  3|2013|    3|  1|   2|   7|   7|  5| 10|300| 73|-1.1|1023.5|             -18.2|   0|          NNW|      5.6|Aotizhongxin|
|  4|2013|    3|  1|   3|   6|   6| 11| 11|300| 72|-1.4|1024.5|             -19.4|   0|           NW|      3.1|Aotizhongxin|
|  5|2013|    3|  1|   4|   3|   3| 12| 12|300| 72|  -2|1025.2|             -19.5|   0|            N|        2|Aotizhongxin|


In [6]:
new_rename_df.count()

35064

In [7]:
new_rename_df.filter( (new_rename_df["WindDirection"] == 'NA')).show()

+-----+----+-----+---+----+----+----+---+---+----+---+----------------+-------+------------------+----+-------------+---------+------------+
|   No|year|month|day|hour|PM25|PM10|SO2|NO2|  CO| O3|            TEMP|   PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+-----+----+-----+---+----+----+----+---+---+----+---+----------------+-------+------------------+----+-------------+---------+------------+
| 6389|2013|   11| 22|   4|  82|  74| 15| 75|1300| NA|            -3.2| 1020.5|                -4|   0|           NA|      0.1|Aotizhongxin|
|11719|2014|    7|  2|   6|  44|  44|  2| 16| 800|105|            21.5|  998.2|              21.5| 0.1|           NA|      0.2|Aotizhongxin|
|13413|2014|    9| 10|  20|  56| 141| 13| 81| 700| 99|            21.7|   1008|              15.8|   0|           NA|        0|Aotizhongxin|
|16749|2015|    1| 27|  20|  26|  42| 12| 78|1300|  2|              NA|     NA|                NA|  NA|           NA|       NA|Aotizhongxin|
|17264|2015| 

In [8]:
from pyspark.sql.functions import col, when

In [9]:
KK = new_rename_df
for i in new_rename_df.columns:
    Kitty = KK.withColumn(i,when((col(i)=='NA'),None).otherwise(col(i)))
    KK = Kitty

In [10]:
after_delete = KK.na.drop()

In [9]:
after_delete.show()

+---+----+-----+---+----+----+----+---+---+---+---+----+------+------------------+----+-------------+---------+------------+
| No|year|month|day|hour|PM25|PM10|SO2|NO2| CO| O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+---+----+-----+---+----+----+----+---+---+---+---+----+------+------------------+----+-------------+---------+------------+
|  1|2013|    3|  1|   0|   4|   4|  4|  7|300| 77|-0.7|  1023|             -18.8|   0|          NNW|      4.4|Aotizhongxin|
|  2|2013|    3|  1|   1|   8|   8|  4|  7|300| 77|-1.1|1023.2|             -18.2|   0|            N|      4.7|Aotizhongxin|
|  3|2013|    3|  1|   2|   7|   7|  5| 10|300| 73|-1.1|1023.5|             -18.2|   0|          NNW|      5.6|Aotizhongxin|
|  4|2013|    3|  1|   3|   6|   6| 11| 11|300| 72|-1.4|1024.5|             -19.4|   0|           NW|      3.1|Aotizhongxin|
|  5|2013|    3|  1|   4|   3|   3| 12| 12|300| 72|  -2|1025.2|             -19.5|   0|            N|        2|Aotizhongxin|


In [12]:
after_delete.write.csv("../DataSet/outliter.csv")

In [13]:
df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("../DataSet/outliter.csv")
df_with_schema.show()

+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+
| No|year|month|day|hour|PM25|PM10| SO2| NO2|   CO|  O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+
|  2|2013|    3|  1|   1| 8.0| 8.0| 4.0| 7.0|300.0|77.0|-1.1|1023.2|             -18.2| 0.0|            N|      4.7|Aotizhongxin|
|  3|2013|    3|  1|   2| 7.0| 7.0| 5.0|10.0|300.0|73.0|-1.1|1023.5|             -18.2| 0.0|          NNW|      5.6|Aotizhongxin|
|  4|2013|    3|  1|   3| 6.0| 6.0|11.0|11.0|300.0|72.0|-1.4|1024.5|             -19.4| 0.0|           NW|      3.1|Aotizhongxin|
|  5|2013|    3|  1|   4| 3.0| 3.0|12.0|12.0|300.0|72.0|-2.0|1025.2|             -19.5| 0.0|            N|      2.0|Aotizhongxin|
|  6|2013|    3|  1|   5| 5.0| 5.0|18.0|18.0|400.0|66.0|-2.2|1025.6|             -19.6| 0.

In [14]:
df_with_schema.printSchema()

root
 |-- No: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- PM25: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- O3: float (nullable = true)
 |-- TEMP: float (nullable = true)
 |-- PRES: float (nullable = true)
 |-- DewPointTempeature: float (nullable = true)
 |-- RAIN: float (nullable = true)
 |-- WindDirection: string (nullable = true)
 |-- WindSpend: float (nullable = true)
 |-- station: string (nullable = true)



In [15]:
df_with_schema.count()

31814

In [16]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

In [17]:
cols = ['PM25','SO2','NO2','CO','O3','TEMP','PRES','DewPointTempeature','RAIN','WindSpend']
bounds = {}

for col in cols:
    quantiles = df_with_schema.approxQuantile(
        col, [0.25, 0.75], 0.05
    )
    
    IQR = quantiles[1] - quantiles[0]
    
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
    ]

In [18]:
bounds

{'CO': [-850.0, 2750.0],
 'DewPointTempeature': [-40.34999942779541, 47.249999046325684],
 'NO2': [-36.5, 143.5],
 'O3': [-94.0, 178.0],
 'PM25': [-97.0, 223.0],
 'PRES': [980.7999572753906, 1041.6000061035156],
 'RAIN': [0.0, 0.0],
 'SO2': [-19.5, 40.5],
 'TEMP': [-24.0999995470047, 50.29999911785126],
 'WindSpend': [-0.8999999165534973, 3.8999997973442078]}

In [38]:
outliers = df_with_schema.select(*['No'] + [
    (
        (df_with_schema[c] < bounds[c][0]) | (df_with_schema[c] > bounds[c][1])
    ).alias(c + '_0') for c in cols
])

outliers.show()

+---+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
| No|PM25_0|SO2_0|NO2_0| CO_0| O3_0|TEMP_0|PRES_0|DewPointTempeature_0|RAIN_0|WindSpend_0|
+---+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
|  2| false|false|false|false|false| false| false|               false| false|       true|
|  3| false|false|false|false|false| false| false|               false| false|       true|
|  4| false|false|false|false|false| false| false|               false| false|      false|
|  5| false|false|false|false|false| false| false|               false| false|      false|
|  6| false|false|false|false|false| false| false|               false| false|      false|
|  7| false|false|false|false|false| false| false|               false| false|      false|
|  8| false|false|false|false|false| false| false|               false| false|      false|
|  9| false|false|false|false|false| false| false|               false| false|       true|

In [39]:
df_outliers = df_with_schema.join(outliers, on='No')
df_outliers.show()

+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
| No|year|month|day|hour|PM25|PM10| SO2| NO2|   CO|  O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|PM25_0|SO2_0|NO2_0| CO_0| O3_0|TEMP_0|PRES_0|DewPointTempeature_0|RAIN_0|WindSpend_0|
+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
|  2|2013|    3|  1|   1| 8.0| 8.0| 4.0| 7.0|300.0|77.0|-1.1|1023.2|             -18.2| 0.0|            N|      4.7|Aotizhongxin| false|false|false|false|false| false| false|               false| false|       true|
|  3|2013|    3|  1|   2| 7.0| 7.0| 5.0|10.0|300.0|73.0|-1.1|1023.5|             -18.2| 0.0|          NNW|      5.6|Aotizhongxin| false|fals

In [41]:
df_outliers.filter('PM25_0' or 'SO2_0' or 'NO2_0' or  'CO_0'or 'O3_0' or 'TEMP_0' or 'PRES_0' or 'DewPointTempeature_0' or 'RAIN_0' or 'WindSpend_0').count()

2145

In [66]:
check  = df_outliers.filter(~df_outliers['PM25_0'] & 
                   ~df_outliers['SO2_0'] & 
                   ~df_outliers['NO2_0'] & 
                   ~df_outliers['CO_0'] & 
                   ~df_outliers['O3_0'] & 
                   ~df_outliers['TEMP_0'] & 
                   ~df_outliers['PRES_0'] &
                   ~df_outliers['DewPointTempeature_0']&
                   ~df_outliers['RAIN_0']&
                   ~df_outliers['WindSpend_0'])

In [72]:
check.count()

21220

In [73]:
check.show()

+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
| No|year|month|day|hour|PM25|PM10| SO2| NO2|   CO|  O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|PM25_0|SO2_0|NO2_0| CO_0| O3_0|TEMP_0|PRES_0|DewPointTempeature_0|RAIN_0|WindSpend_0|
+---+----+-----+---+----+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+------+-----+-----+-----+-----+------+------+--------------------+------+-----------+
|  4|2013|    3|  1|   3| 6.0| 6.0|11.0|11.0|300.0|72.0|-1.4|1024.5|             -19.4| 0.0|           NW|      3.1|Aotizhongxin| false|false|false|false|false| false| false|               false| false|      false|
|  5|2013|    3|  1|   4| 3.0| 3.0|12.0|12.0|300.0|72.0|-2.0|1025.2|             -19.5| 0.0|            N|      2.0|Aotizhongxin| false|fals

In [77]:
afterDrop = check.drop(*["PM25_0","SO2_0",'NO2_0','CO_0','O3_0','TEMP_0','PRES_0','DewPointTempeature_0','RAIN_0','WindSpend_0',"No","PM10"])

In [78]:
afterDrop.show()

+----+-----+---+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+
|year|month|day|hour|PM25| SO2| NO2|   CO|  O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+----+-----+---+----+----+----+----+-----+----+----+------+------------------+----+-------------+---------+------------+
|2013|    3|  1|   3| 6.0|11.0|11.0|300.0|72.0|-1.4|1024.5|             -19.4| 0.0|           NW|      3.1|Aotizhongxin|
|2013|    3|  1|   4| 3.0|12.0|12.0|300.0|72.0|-2.0|1025.2|             -19.5| 0.0|            N|      2.0|Aotizhongxin|
|2013|    3|  1|   5| 5.0|18.0|18.0|400.0|66.0|-2.2|1025.6|             -19.6| 0.0|            N|      3.7|Aotizhongxin|
|2013|    3|  1|   6| 3.0|18.0|32.0|500.0|50.0|-2.6|1026.5|             -19.1| 0.0|          NNE|      2.5|Aotizhongxin|
|2013|    3|  1|   7| 3.0|19.0|41.0|500.0|43.0|-1.6|1027.4|             -19.1| 0.0|          NNW|      3.8|Aotizhongxin|
|2013|    3|  1|   9| 3.0|12.0|2

In [81]:
afterDrop.createOrReplaceTempView('dataset')
results = spark.sql("SELECT * FROM dataset WHERE year > 2014")
results.show()
print("Size of dataset",results.count())

+----+-----+---+----+----+----+----+------+----+----+------+------------------+----+-------------+---------+------------+
|year|month|day|hour|PM25| SO2| NO2|    CO|  O3|TEMP|  PRES|DewPointTempeature|RAIN|WindDirection|WindSpend|     station|
+----+-----+---+----+----+----+----+------+----+----+------+------------------+----+-------------+---------+------------+
|2015|    1|  1|   4|12.0|14.0|33.0| 700.0|24.0|-2.0|1027.0|             -23.2| 0.0|          ENE|      1.4|Aotizhongxin|
|2015|    1|  1|   5|12.0|10.0|34.0| 600.0|23.0|-5.0|1030.0|             -23.5| 0.0|            N|      1.1|Aotizhongxin|
|2015|    1|  1|   6| 6.0| 4.0|19.0| 500.0|34.0|-6.0|1029.0|             -23.8| 0.0|           NE|      1.6|Aotizhongxin|
|2015|    1|  1|   7|11.0| 7.0|25.0| 500.0|28.0|-7.0|1030.0|             -24.2| 0.0|           NE|      2.0|Aotizhongxin|
|2015|    1|  1|   8|12.0|11.0|49.0| 600.0|10.0|-6.0|1028.0|             -23.8| 0.0|            E|      0.9|Aotizhongxin|
|2015|    1|  1|  10|15.

In [44]:
remove_outliers.save('../DataSet/check2.csv',header = 'true')



AttributeError: 'DataFrame' object has no attribute 'save'

In [69]:
check \
  .write \
  .mode('overwrite') \
  .option('header', 'true') \
  .csv('../DataSet/views.csv')